In [1]:
import sys

sys.argv = [""]
del sys

In [2]:
import argparse
import os
from copy import deepcopy

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from inpaint.data import PlacesDataset
from inpaint.utils import random_bbox_mask, random_ff_mask
from inpaint.utils.metrics import psnr
from inpaint.utils.metrics import ssim
import matplotlib.pyplot as plt

import random

In [4]:
from inpaint.core.generator import GatedGenerator
from inpaint.tools import Evaluate

In [3]:
def flipCoin():
    f = random.random()
    return True if f < 0.5 else False

In [5]:
def create_cfg(args):
    parser = argparse.ArgumentParser()
    for key, val in args.items():
        parser.add_argument(key, default=val)

    cfg = parser.parse_args()
    return cfg

In [26]:
args = {}
# model
args["--in_channels"] = 4
args["--out_channels"] = 3
args["--latent_channels"] = 64
args["--pad_type"] = "zero"
args["--activation"] = "elu"
args["--norm_d"] = "none"
args["--norm_g"] = "none"
args["--init_type"] = "kaiming"
args["--init_gain"] = 0.02
args["--use_cuda"] = True

# dataset
args["--val_ds_dir"] = '../../samples/Places365'
args["--crop_size"] = (256, 256)
args["--batch_size"] =  1
args["--num_workers"] = 1
args["--device_id"] = 0
args["--val_steps"] = 10

In [22]:
cfg = create_cfg(args)

In [23]:
val_ds = PlacesDataset(path_dir = cfg.val_ds_dir, transform_config=('to_tensor', 'center_crop', 'norm'), crop_size=cfg.crop_size)
val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=True, pin_memory=True, num_workers=cfg.num_workers)

Total images:  24


In [24]:
print(f"Total validation images: {len(val_ds)}")

Total validation images: 24


In [25]:
eval = Evaluate(cfg, val_loader)

In [ ]:
eval.evaluate("../../../../experiments/inpaint/ckpts/exp1")